# Worksheet 08

Name:  Aidan Clark

UID: U01817265

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [13]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.345646458289913, 3.344686709817138, 5.769146039387756, 5.938297231660819, 6.273314986703041, 4.9130037506300805, 5.756818514873793, 3.4482998959435553, 6.344382598958948, 5.630704872560143]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [14]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[8.460635140762712, 6.443005128668924, 7.996215667886662, 7.175117006599108, 8.476174511709052, 7.831009600931495, 7.216737976260395, 7.83066005576512, 8.820593730979837, 6.839089920487453]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [15]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.630704872560143, 6.344382598958948, 6.839089920487453, 3.4482998959435553, 5.756818514873793, 8.820593730979837, 4.9130037506300805, 6.273314986703041, 7.83066005576512, 5.938297231660819]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Prior probability of being from s1 = 0.5

Prior probability of being from s2 = 0.5

Mean of s1 = 5

Variance of s1 = 1

Mean of s2 = 8

Variance of s2 = 1


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [16]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
#Take everything in data that was assigned to s1
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.630704872560143, 6.344382598958948, 3.4482998959435553, 5.756818514873793, 4.9130037506300805, 6.273314986703041, 5.938297231660819]
[6.839089920487453, 8.820593730979837, 7.83066005576512]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 5.472117407332911,  mean_2 = 7.83011456907747
var_1 = 0.8763911228001973,  var_2 = 0.6543930406105031


These values seem reasonably close. Our probabilities are off, but the means are pretty close to the actual means, and the variances are pretty close as well.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [18]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i) #List of probabilities, one for each of the n datapoints
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], (var[j]))**(1/2)) #Changed to sqrt because norm.pdf takes stdev not var
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    #pdf_i[0] is prob of point p given species_0

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.630704872560143
probability of observing that point if it came from cluster 0 =  0.6691921397461968
probability of observing that point if it came from cluster 1 =  0.04635100480838295
point =  6.344382598958948
probability of observing that point if it came from cluster 0 =  0.5266866629010593
probability of observing that point if it came from cluster 1 =  0.21521423793754219
point =  6.839089920487453
probability of observing that point if it came from cluster 0 =  0.3672465058980412
probability of observing that point if it came from cluster 1 =  0.4400745086738962
point =  3.4482998959435553
probability of observing that point if it came from cluster 0 =  0.17787545597923826
probability of observing that point if it came from cluster 1 =  1.0580012633393337e-05
point =  5.756818514873793
probability of observing that point if it came from cluster 0 =  0.6571251725171556
probability of observing that point if it came from cluster 1 =  0.06348607022925747
point =  8.82059

In general, it looks like these estimates are pretty good-- points that are closer to 5 have higher estimated probabilities of being from s1 and points that are closer to 8 have higher probabilities of being from s2. Picking the higher probability for each point, we predict that s1 was at least the source of 5.63, 6.34, 3.45, 5.76, 4.91, and 6.27 and that s2 was at least the source of 6.84, 8.82, and 7.83. These estimates seem pretty reasonable, although potentially there are some points in the middle that are incorrectly labeled s1.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [19]:
#Apply formulas from slide
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 5.669544537869476,  mean_2 = 7.777900003410774
var_1 = 1.212462047349182,  var_2 = 1.0050482163905166


Our incorrect probabilities have stayed the same, but the mean for s2 improved slightly (went from 7.83 to 7.77) and both of our variances are now pretty close to the actual variances (increased from 0.88 and 0.6). Our estimates would certainly benefit from further iterations, however.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [31]:
#Repeating the code from before
prob_s0_x = [] # P(S_0 | X_i) #List of probabilities, one for each of the n datapoints
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], (var[j]))**(1/2)) #Changed to sqrt because norm.pdf takes stdev not var
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    #pdf_i[0] is prob of point p given species_0

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x )

    probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()



point =  5.630704872560143
probability of observing that point if it came from cluster 0 =  0.5734684669536233
probability of observing that point if it came from cluster 1 =  0.2012818685076826
point =  6.344382598958948
probability of observing that point if it came from cluster 0 =  0.5308676826661284
probability of observing that point if it came from cluster 1 =  0.3788624362108011
point =  6.839089920487453
probability of observing that point if it came from cluster 0 =  0.45456658954346957
probability of observing that point if it came from cluster 1 =  0.5065561046569219
point =  3.4482998959435553
probability of observing that point if it came from cluster 0 =  0.2478671172694346
probability of observing that point if it came from cluster 1 =  0.006088298198833917
point =  5.756818514873793
probability of observing that point if it came from cluster 0 =  0.5728730750840495
probability of observing that point if it came from cluster 1 =  0.22924894496724646
point =  8.820593730

These seem like more reasonable probabilities. Points like 3.45 that are clearly from s1 have near zero probabilities of being from s2. Points in the middle like 6.34 and 6.84 have sizable probabilities for s1 and s2, with the higher probability being that of the distribution whose mean is closest to the point

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [32]:
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    assignment = ""
    if p[1] > p[2]:
        assignment = "0"
    else:
        assignment = "1"

    print("Point ", p[0], " assigned to ", assignment)
    print()
    

Point  5.630704872560143  assigned to  0

Point  6.344382598958948  assigned to  0

Point  6.839089920487453  assigned to  0

Point  3.4482998959435553  assigned to  0

Point  5.756818514873793  assigned to  0

Point  8.820593730979837  assigned to  1

Point  4.9130037506300805  assigned to  0

Point  6.273314986703041  assigned to  0

Point  7.83066005576512  assigned to  1

Point  5.938297231660819  assigned to  0

